# Hypothesis Testing with Insurance Data (Core)
For this assignment, we will be working with the US Health Insurance Dataset from Kaggle.

We have been asked to use our hypothesis testing skills to answer the following questions:

- Q1. Do smokers have higher insurance charges than non-smokers?
- Q2. Are men more likely to smoke than women?
- Q3. Do different regions have different charges, on average?

### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

### Load the data

In [2]:
df = pd.read_csv('Data/insurance - insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


# Q1. Do smokers have higher insurance charges than non-smokers?

## 1. State your Null Hypothesis and Alternative Hypothesis

- Null Hypothesis: There is no difference in insurance charges between smokers and non-smokers.
- Alternative Hypothesis: There is a significant difference in insurance charges between smokers and non-smokers.

## 2. Select the correct test according to the data type and number of samples

- Numeric
- Two groups/ samples
- 2-sample T-Test will be run

## 3. Test the assumptions of your selected test.

### Test Assumptions
- No significant outliers
- Normality
- Equal Variance

## 4. Execute the selected test, or the alternative test (if you do not meet the assumptions)

## 5. Interpret your p-value and reject or fail to reject your null hypothesis 


## 6. Show a supporting visualization that helps display the result


## Q2. Are men more likely to smoke than women?

### 1. State your Null Hypothesis and Alternative Hypothesis

- Null Hypothesis:
- Alternative Hypothesis:

### 2. Select the correct test according to the data type and number of samples

### 3. Test the assumptions of your selected test.

### 4. Execute the selected test, or the alternative test (if you do not meet the assumptions)


### 5. Interpret your p-value and reject or fail to reject your null hypothesis 


### 6. Show a supporting visualization that helps display the result


## Q3. Do different regions have different charges, on average?

### 1. State your Null Hypothesis and Alternative Hypothesis

- Null Hypothesis:
- Alternative Hypothesis:

### 2. Select the correct test according to the data type and number of samples

### 3. Test the assumptions of your selected test.

### 4. Execute the selected test, or the alternative test (if you do not meet the assumptions)


### 5. Interpret your p-value and reject or fail to reject your null hypothesis 


### 6. Show a supporting visualization that helps display the result
